In [1]:
import microtubulepkg as mp

Loading BokehJS ...

# Analysis of Microtubule Catastrophe

##### Rosita Fu, Theresa Marlin, Erika Salzman

In this notebook, we will be analyzing data on the time to catastrophe of microtubules in solutions containing different total concentrations of tubulin labeled with fluorescent markers.

(might want to add more to this)

----

### Visualizing the data

First, we will import our data. The `setup` function intakes data as a CSV file as well as the concentrations of tubulin used and outputs a tidy dataframe containing the data, previewed below.

The raw data used is available in a CSV file within this repository.

In [2]:
df = mp.setup('./gardner_mt_catastrophe_only_tubulin.csv', ['7 uM', '9 uM', '10 uM', '12 uM', '14 uM'])
df.head()

,index,concentration,catastrophe time,concentration (int)
0,0,7 uM,35.0,7
1,1,7 uM,45.0,7
2,2,7 uM,50.0,7
3,3,7 uM,50.0,7
4,4,7 uM,55.0,7


To help visualize this data, we will plot ECDFs and box plots of the times to catastrophe for the different concentrations of tubulin.

In [3]:
mp.plot_ecdf(df)

In [4]:
mp.plot_box(df)

### Modeling microtubule catastrophe

We would like to formulate a generative model for this process. Two possible models we will be examining are the following:
1. Gamma distribution. You can learn more about this distribution [here](http://bois.caltech.edu/distribution_explorer/continuous/gamma.html). This distribution contains two parameters: $\alpha$, the number of arrivals, and $\beta$, the rate of arrivals. 
2. Two Poisson processes with different rates of arrival. This distribution also contains two parameters: $\beta_{1}$ and $\beta_{2}$. We will refer to this distribution as the custom distribution.

We will look specifically at the data for time to catastrophe at 12 uM tubulin concentration.

In [5]:
tub_12 = df.loc[df["concentration"] == "12 uM"]
tub_12.head()

,index,concentration,catastrophe time,concentration (int)
1087,2076,12 uM,25.000,12
1088,2077,12 uM,40.000,12
1089,2078,12 uM,40.000,12
1090,2079,12 uM,45.429,12
1091,2080,12 uM,50.000,12


We will first obtain maximum likelihood estimates for the parameters $\alpha$ and $\beta$ of the Gamma distribution.

In [6]:
t = tub_12["catastrophe time"].values
alpha_mle, beta_mle = mp.gamma_mle(t)

print(f'α MLE estimate: {alpha_mle:.3f}')
print(f'β MLE estimate: {beta_mle:.3f}')

α MLE estimate: 2.743
β MLE estimate: 0.007


To visually assess the model, we can plot our data on top of the generative distribution parameterized by the MLE estimates for $\alpha$ and $\beta$. Here, the generative model is plotted in orange, and the data are plotted in blue.

In [7]:
mp.plot_model_gamma(t, alpha_mle, beta_mle)

We can also check to see the variation in MLE estimate for the parameters given our generative model. To do this, we will compute 95% confidence intervals for the MLE estimates of $\alpha$ and $\beta$ by bootstrapping 1000 samples from our model.

In [8]:
mp.bootstrap_CI_gamma(t)

100%|██████████| 1000/1000 [00:08<00:00, 121.55it/s]


α 95% confidence interval: [2.462, 3.095]
β 95% confidence interval: [0.006, 0.008]


These confidence intervals seem reasonable, given our data.

Next, we will make another generative distribution for our data, but using our custom distribution (two Poisson processes) instead. First, we will obtain MLE estimates for $\beta_{1}$ and $\beta_{2}$ for the 12 uM tubulin concentration trial.

Note that delta_beta is the difference between the two $\beta$ values.

In [9]:
beta_1_mle, delta_beta = mp.custom_mle(t)
beta_2_mle = beta_1_mle + delta_beta

print(f'β_1 MLE estimate: {beta_1_mle:.4f}')
print(f'β_2 MLE estimate: {beta_2_mle:.4f}')

β_1 MLE estimate: 0.0048
β_2 MLE estimate: 0.0056


Once again, to assess the model, we will plot our data on top of the generative distribution parameterized by the MLE estimates for $\beta_1$ and $\beta_2$. The generative model is plotted in orange, and the data are plotted in blue.

In [10]:
mp.plot_model_custom(t, beta_1_mle, beta_2_mle)

Visually, it looks like the Gamma distribution is a better fit for our data. We will also use the Akaike information criterion to assess our models.

In [11]:
mp.aic_model_comp(t, alpha_mle, beta_mle, beta_1_mle, beta_2_mle)

,model,alpha,beta,beta 1,beta 2,log like,AIC,AIC weight
0,gamma,2.7435,0.00720879,NaN,NaN,-4637.87,9279.74,1
1,2-step poisson,NaN,NaN,0.00484965,0.00564003,-4712.46,9428.93,0


We see here that the AIC value for the gamma distribution is smaller, indicating that the gamma distribution is indeed a better fit for our data.

With our model chosen, we will make generative models for the other concentrations tested. First, we will generate MLE estimates for $\alpha$ and $\beta$ for each concentration.

In [12]:
mle_data = mp.setup_mle_data(df)
mle_data

,molarity (uM),alpha MLE,beta MLE
0,7,2.443910,0.007550
1,9,2.763583,0.009053
2,10,3.281072,0.009227
3,12,2.743497,0.007209
4,14,3.322737,0.007092


In [13]:
mp.plot_all_concentrations(df, mle_data)